In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import json

# json_file_path = "put your path here"

with open(json_file_path, 'r') as file:
    data = json.load(file)


In [ ]:
def insert_phrase_after_kth_newline(original_string, phrase, k):

    lines = original_string.split('\n')
    lines[k] = lines[k] + phrase
    modified_string = '\n'.join(lines)

    return modified_string

In [ ]:
import re

def extract_dialog_from_path(path):
    match = re.search(r'dialog_(\d+)', path)
    if match:
        return match.group(1)
    else:
        return None

def extract_turn_from_path(path):
    match = re.search(r'turn_(\d+)', path)
    if match:
        return match.group(1)
    else:
        return None

def extract_state_from_path(path):
    match = re.search(r'(\d+).png', path)
    if match:
        return match.group(1)
    else:
        return None

In [ ]:
def dict_to_html(data):

    all_html_list = []
    seen_apps_list = []

    for each_page in data:
        html = ''
        dialog = extract_dialog_from_path(each_page.get("screenshot_history")[-1])
        turn = extract_turn_from_path(each_page.get("screenshot_history")[-1])
        state = extract_state_from_path(each_page.get("screenshot_history")[-1])
        html += f"Dialog {dialog} Turn {turn} State {state}:\n"
        items = each_page.get("items", [])
        app = each_page.get("apps", [])[0]
        action = each_page.get("action")
        if app not in seen_apps_list:
            seen_apps_list.append(app)

        for item_index, each_item in enumerate(items):
            text = each_item.get("text", {}).get("text", "")
            content_desc = each_item.get("text", {}).get("content-desc", "")
            element_type = each_item.get("type", "")
            border = each_item.get("border", [])

            if element_type == "android.widget.TextView":
                html += f'<element{item_index}:<TextView>{text}</TextView>\n'
            elif element_type == "android.widget.ImageView":
                html += f'<element{item_index}:<ImageView>{content_desc}</ImageView>\n'
            elif element_type == "android.widget.LinearLayout":
                html += f'<element{item_index}:<LinearLayout>{content_desc}</LinearLayout>\n'
            elif element_type == "android.widget.ImageButton":
                html += f'<element{item_index}:<ImageButton>{content_desc}</ImageButton>\n'
            elif element_type == "android.widget.Button":
                html += f'<element{item_index}:<Button>{text}</Button>\n'
            elif element_type == "android.widget.EditText":
                html += f'<element{item_index}:<EditText>{text}</EditText>\n'
            elif element_type == "android.widget.Switch":
                html += f'<element{item_index}:<Switch>{text}</Switch>\n'
            elif element_type == "android.view.View":
                html += f'<element{item_index}:<View>{content_desc}</View>\n'
            elif element_type == "android.view.ViewGroup":
                html += f'<element{item_index}:<ViewGroup>{border}</ViewGroup>\n' # "text": null, "content-desc": ""
            elif element_type == "android.widget.CheckedTextView":
                html += f'<element{item_index}:<CheckedTextView>{text}</CheckedTextView>\n'
            elif element_type == "android.widget.RelativeLayout":
                html += f'<element{item_index}:<RelativeLayout>{content_desc}</RelativeLayout>\n'
            elif element_type == "android.webkit.WebView":
                html += f'<element{item_index}:<WebView>{border}</WebView>\n' # "text": null, "content-desc": ""
            elif element_type == "android.widget.android.widget.Image":
                html += f'<element{item_index}:<Image>{text}</Image>\n'
            elif element_type == "android.widget.ToggleButton":
                html += f'<element{item_index}:<ToggleButton>{content_desc}</ToggleButton>\n'
            elif element_type == "android.widget.ListView":
                html += f'<element{item_index}:<ListView>{content_desc}</ListView>\n'
            elif element_type == "android.widget.FrameLayout":
                html += f'<element{item_index}:<FrameLayout>{border}</FrameLayout>\n' # "text": null, "content-desc": ""
            elif element_type == "android.widget.RadioButton":
                html += f'<element{item_index}:<RadioButton>{text}</RadioButton>\n'
            elif element_type == "android.widget.CheckBox":
                html += f'<element{item_index}:<CheckBox>{text}</CheckBox>\n'

        input_content = each_page.get("input")
        html += f'<element{item_index+1}:<Input>{input_content}</Input>'
        if input_content != None:
            if extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == dialog and extract_turn_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == turn:
                html += f' // on input {input_content}, go to Dialog {dialog} Turn {turn} State {int(state)+1}\n'
            elif extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == dialog and extract_turn_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) != turn:
                html += f' // on input {input_content}, go to Dialog {dialog} Turn {int(turn)+1} State {0}\n'
            elif extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) != dialog:
                html += f' // on input {input_content}, go to Dialog {int(dialog)+1} Turn {0} State {0}\n'
        else:
            html += '\n'

        scroll_dir = each_page.get("scroll")
        html += f'<element{item_index+2}:<Scroll>{scroll_dir}</Scroll>'
        if scroll_dir != None:
            if extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == dialog and extract_turn_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == turn:
                html += f' // on scroll {scroll_dir}, go to Dialog {dialog} Turn {turn} State {int(state)+1}\n'
            elif extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == dialog and extract_turn_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) != turn:
                html += f' // on scroll {scroll_dir}, go to Dialog {dialog} Turn {int(turn)+1} State {0}\n'
            elif extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) != dialog:
                html += f' // on scroll {scroll_dir}, go to Dialog {int(dialog)+1} Turn {0} State {0}\n'
        else:
            html += '\n'

        if each_page.get("target") != None:
            if extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == dialog and extract_turn_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == turn:
                comment_insert = f' // on {action}, go to Dialog {dialog} Turn {turn} State {int(state)+1}'
            elif extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) == dialog and extract_turn_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) != turn:
                comment_insert = f' // on {action}, go to Dialog {dialog} Turn {int(turn)+1} State {0}'
            elif extract_dialog_from_path(data[data.index(each_page) + 1].get("screenshot_history")[-1]) != dialog:
                comment_insert = f' // on {action}, go to Dialog {int(dialog)+1} Turn {0} State {0}'
            html = insert_phrase_after_kth_newline(html, comment_insert, each_page.get("target")+1)

        all_html_list.append(html)

    return all_html_list

In [ ]:
all_html_list = dict_to_html(data)

In [ ]:
# processed_htmls_mytraindata_file_path = "put your path here"

with open(processed_htmls_mytraindata_file_path, 'w') as file:
    for each_html_string in all_html_list:
        file.write(each_html_string + "\n")


In [ ]:
# processed_htmls_mytraindata_file_path = "put your path here"

with open(processed_htmls_mytraindata_file_path, 'r') as file:
    lines = file.readlines()
    for line in lines:
        print(line.strip())

In [ ]:
!pip install openai==0.28

In [ ]:
def query_gpt(prompt, model_name="gpt-3.5-turbo"):
    import time
    import openai
    # openai.api_base = 'put your api base link here2'
    # openai.api_key = 'put your api key here'

    response = openai.ChatCompletion.create(
        model=model_name,
        messages=[
            # {"role": "system", "content": "You are a personal agent running in a smartphone."},
            {"role": "user", "content": prompt},
        ],
        timeout=15
    )
    return response["choices"][0]["message"]["content"]

In [ ]:
prompt_prefix = (f'You are a mobile app testing expert. Given the description of the pages of an app, '
                     f'you can understand '
              f'the functions of the app and write useful functional tests. \nNow suppose you are testing an app named '
              f'"{app_name}". The states in the app are:\n\n')
prompt_end = ('Please list some important use cases of this app. Each use case should be in the format: \ntask '
                'description: <a specific task that can be executed with this app, be specific about the parameters '
                '(names, accounts, numbers, addresses, etc)>. Example: Find messages with Alice; Send a text '
                'message "Morning" to Alice; etc. \naction sequence: <a list of GUI actions to complete the task. '
                'Each action includes action type (touch, long_touch, select, scroll down, set_text, etc.), '
                'target element, and an optional input text (for set_text action). Example: touch {state 3 element '
                '0 <button>Rename note</button>}> \nYour answer should be a list, each element of which should be a '
                'dict: {"task": <a task in the string>, "actions": <an action sequence to complete the task>}. For '
                'example:  [{"task": <task1>, "actions": <action sequence1>}, {"task": <task2>, "actions": <action '
                'sequence2>} ...]')

In [ ]:
def get_first_dts_line(html):

    newline_index = html.find('\n')

    res = html[:newline_index]

    return res

def get_dialog_and_turn_number(dts_line):


    d_match = re.search(r'Dialog (\d+)', dts_line)
    t_match = re.search(r'Turn (\d+)', dts_line)

    if d_match and t_match:
        return d_match.group(1), t_match.group(1)
    else:
        print("Dialog and Turn info is not completed")
        return None



In [ ]:
all_apps_dict = dict()
last_dialog = 0
last_turn = 0
this_turn_htmls_str = ''
for html_index in range(len(all_html_list)):
    app_name = data[html_index].get("apps", [])[0][4:]
    if app_name not in all_apps_dict:
        all_apps_dict[app_name] = dict()
    dialog, turn = get_dialog_and_turn_number(get_first_dts_line(all_html_list[html_index]))
    if dialog == last_dialog and turn == last_turn:
        this_turn_htmls_str += all_html_list[html_index] + '\n'
    else:
        all_apps_dict[app_name][len(all_apps_dict[app_name])] = prompt_prefix + this_turn_htmls_str + prompt_end
        this_turn_htmls_str = all_html_list[html_index] + '\n'
    last_dialog = dialog
    last_turn = turn


In [ ]:
# save the processed data in terms of each turn for each app
# each (screen) state is represented in html format

import json

# generate_all_tasks_prompts_file_path = "put your path here"

with open(generate_all_tasks_prompts_file_path, 'w') as json_file:
    json.dump(all_apps_dict, json_file, indent=4)